# Exporting models for C++

In [1]:
%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import random
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
import requests
from io import BytesIO
import cv2

Fetch an example image

In [18]:
response = requests.get("https://github.com/pytorch/hub/raw/master/images/dog.jpg")
file = BytesIO(response.content)
file.seek(0)
image = cv2.imdecode(np.frombuffer(file.read(), dtype=np.uint8), cv2.IMREAD_COLOR)
image = cv2.resize(image, (224, 224))
cv2.imwrite("../data/dog.jpg", image)

True

## Setting up a model

In [3]:
import torch
import torchvision
from torchvision import transforms

Create a feature extractor without the classifier

In [19]:
model = torchvision.models.mobilenet_v2(pretrained=True)
feature_extractor = torch.nn.Sequential(
    model._modules.get("features"), torch.nn.AdaptiveAvgPool2d(1)
)
# Set model into evaluation mode
feature_extractor = feature_extractor.eval()

To use the model with the `torchlib` C++ library, it must first be converted to TorchScript. If your model involves conditional statements written in Python, you must use `torch.jit.script`.

In [23]:
feature_extractor_jited = torch.jit.script(feature_extractor)
feature_extractor_jited.save("../models/MobileNetV2-FeatureExtractor.pt")

Alternatively, you can trace the model with an example image if your model is just pure tensor operations:

In [24]:
resnet18 = torchvision.models.resnet18()
example = torch.rand(1, 3, 224, 224)
resnet18_traced = torch.jit.trace(resnet18, example)
resnet18_traced.save("../models/ResetNet18.pt")

In [21]:
image = image.astype(np.float32)
image = torch.tensor(image).reshape(1, 3, 224, 224)

We can even use the JITed model to run predictions:

In [27]:
feature_extractor_jited(image).detach().numpy()[0, 0:5, 0, 0]

array([0.36407703, 0.        , 0.29994395, 0.19936535, 0.16753083],
      dtype=float32)

Refer to the [PyTorch CPP / TorchScript](https://github.com/ismailuddin/PyTorch-CPP-TorchScript) repository for details on how to use the exported model. You can compare the outputs in Python to those from the C++ run predictions.